In [1]:
import numpy as np
import pandas as pd
import sklearn

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import average_precision_score, f1_score

from scipy.optimize import minimize_scalar

In [12]:
def get_training_week_data_filename_QD(week_number, simulated_org_number):
    head_folder_name = "C:/Users/Mimran/Google Drive/RCP 13 and RCP14/RCP13 and RCP14/July 4 Population"
    full_filename = "{}/{}/org ({}).csv".format(head_folder_name, 
                                                     week_number, 
                                                     simulated_org_number)
    return full_filename

In [13]:
from pathlib import Path
for week_number in range(5, 33):
    print("can find week {}:{}".format(week_number, 
                                       Path(get_training_week_data_filename_QD(week_number, 9)).exists()))

can find week 5:True
can find week 6:True
can find week 7:True
can find week 8:True
can find week 9:True
can find week 10:True
can find week 11:True
can find week 12:True
can find week 13:True
can find week 14:True
can find week 15:True
can find week 16:True
can find week 17:True
can find week 18:True
can find week 19:True
can find week 20:True
can find week 21:True
can find week 22:True
can find week 23:True
can find week 24:True
can find week 25:True
can find week 26:True
can find week 27:True
can find week 28:True
can find week 29:True
can find week 30:True
can find week 31:True
can find week 32:True


In [14]:
def read_training_week_data_QD(week_number, simulated_org_number):
    full_filename = get_training_week_data_filename_QD(week_number, simulated_org_number)
    week_df = pd.read_csv(full_filename, usecols=list(range(1,67))) # Note this assumes similar order of users everywhere
    week_df.replace([-np.inf,np.inf], np.nan, inplace=True) #(no matching of users is necessary *under this assumption*)
    return week_df.dropna()

In [15]:
detector_names = read_training_week_data_QD(50, 5).columns.values.tolist()

In [16]:
detector_names

['Target',
 'X001a',
 'X001b',
 'X001c',
 'X014a',
 'X015a',
 'X021a',
 'X021d',
 'X021e',
 'X021f',
 'X021g',
 'X021h',
 'X021i',
 'X021j',
 'X022a',
 'X022d',
 'X022e',
 'X022f',
 'X022g',
 'X022h',
 'X022i',
 'X022j',
 'X027a',
 'X027d',
 'X027e',
 'X027f',
 'X027g',
 'X027h',
 'X027i',
 'X027j',
 'X028a',
 'X028d',
 'X028e',
 'X028f',
 'X028g',
 'X028h',
 'X028i',
 'X028j',
 'X029a',
 'X030a',
 'X031a',
 'X032a',
 'X033a',
 'X034a',
 'X035a',
 'X036a',
 'X037a',
 'X038a',
 'X039a',
 'X040a',
 'X041a',
 'X042a',
 'X043a',
 'X044a',
 'X045a',
 'X046a',
 'X047a',
 'X048a',
 'X049a',
 'X050a',
 'X051a',
 'X052a',
 'X053a',
 'X058a',
 'X059a',
 'X060a']

In [17]:
np.random.seed(662017)

In [2]:
def read_training_org_data(simulated_org_number, first_full_week, last_full_week):
    all_full_week_dfs = [read_training_week_data_QD(week_number, simulated_org_number) for week_number in range(first_full_week, last_full_week + 1)]
    return pd.concat(all_full_week_dfs).dropna()

In [3]:
def split_training_data(sample_training_data_df):
    Ones_x = sample_training_data_df[sample_training_data_df['Target'] == 1].drop('Target', 1)
    Ones_y = sample_training_data_df[sample_training_data_df['Target'] == 1]['Target']
    Zeros = sample_training_data_df[sample_training_data_df['Target'] == 0]
    Z12_x, Z3_x, Z12_y, Z3_y = train_test_split(Zeros.drop('Target', 1), Zeros['Target'],
                                                test_size = 0.3)
    #print(Ones_x.shape)
    scaler = StandardScaler().fit(Z12_x)
    Ones_x = scaler.transform(Ones_x)
    Z12_x = scaler.transform(Z12_x)
    Z3_x = scaler.transform(Z3_x)
    return Z12_x, Z3_x, Z12_y.values, Z3_y.values, Ones_x, Ones_y.values, scaler

In [71]:
from sklearn.neural_network import MLPRegressor

In [95]:
def RCP13_Algorithm_1(Z12_x, Z3_x, Z12_y, Z3_y, Ones_x, Ones_y, iter_per_k = 1):
    k_average_areas = {}
    for k_to_test in range(2, 65):
        print(k_to_test)
        k_areas = np.zeros(iter_per_k)
        for iteration_num in range(iter_per_k):
            n_net = MLPRegressor(hidden_layer_sizes=(k_to_test,), 
                                  activation = 'tanh', 
                                  solver = 'sgd', max_iter = 250, early_stopping = True)
            n_net.fit(Z12_x, Z12_y)
            model_prediction_output = n_net.predict(np.vstack([Z3_x, Ones_x]))
            k_areas[iteration_num] = average_precision_score(np.hstack([Z3_y, Ones_y]), model_prediction_output)
            print("current area is {}".format(k_areas[iteration_num]))
        k_average_areas[k_to_test] = k_areas.mean()
        print("k {} had an average area of {}".format(k_to_test, k_average_areas[k_to_test]))
    return max(k_average_areas.keys(), key=(lambda key: k_average_areas[key]))

In [4]:
some_df = read_training_org_data(40, 5, 33)

NameError: name 'read_training_week_data_QD' is not defined

In [292]:
X_train, X_test, y_train, y_test = train_test_split(some_df.drop('Target', 1), some_df['Target'],
                                                test_size = 0.3)

In [299]:
X_train.shape

(60701, 65)

In [293]:
smartClassifier = RandomForestClassifier()
smartClassifier.fit(X_train, y_train)
y_pred = smartClassifier.predict(X_test)

In [294]:
precision_score(y_pred = y_pred, y_true=y_test)

1.0

In [295]:
recall_score(y_pred = y_pred, y_true=y_test)

0.81632653061224492

In [296]:
confusion_matrix(y_pred = y_pred, y_true=y_test)

array([[25966,     0],
       [    9,    40]])

In [298]:
y_test.sum()

49

In [115]:
Z12_x, Z3_x, Z12_y, Z3_y, Ones_x, Ones_y, some_scaler = split_training_data(some_df)

In [211]:
some_df['Target'].mean()

0.001372295770100097

In [116]:
from sklearn.tree import DecisionTreeClassifier

In [282]:
tree_for_training_data = DecisionTreeClassifier(max_depth = 3, random_state = 711)
tree_for_training_data.fit(some_df.drop('Target', 1), some_df['Target'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=711, splitter='best')

In [189]:
tree_for_training_data.score(some_df.drop('Target', 1), some_df['Target'])

0.99944646893306888

In [190]:
y_pred = tree_for_training_data.predict(some_df.drop('Target', 1))

In [191]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix

In [192]:
precision_score(y_pred = y_pred, y_true = some_df['Target'])

0.98630136986301364

In [193]:
recall_score(y_pred = y_pred, y_true = some_df['Target'])

0.60504201680672265

In [175]:
confusion_matrix(y_pred = y_pred, y_true = some_df['Target'])

array([[86596,     1],
       [   47,    72]])

In [155]:
from sklearn.dummy import DummyClassifier

In [194]:
dumbClassifier = DummyClassifier(random_state=711, strategy='stratified')
dumbClassifier.fit(some_df.drop('Target', 1), some_df['Target'])
y_pred = dumbClassifier.predict(some_df.drop('Target', 1))

In [195]:
precision_score(y_pred = y_pred, y_true = some_df['Target'])

0.0090090090090090089

In [196]:
recall_score(y_pred = y_pred, y_true = some_df['Target'])

0.0084033613445378148

In [159]:
confusion_matrix(y_pred = y_pred, y_true = some_df['Target'])

array([[86472,   125],
       [  118,     1]])

In [197]:
y_pred.sum()

111

In [198]:
from sklearn.ensemble import RandomForestClassifier

In [216]:
smartClassifier = RandomForestClassifier()
smartClassifier.fit(some_df.drop('Target', 1), some_df['Target'])
y_pred = smartClassifier.predict(some_df.drop('Target', 1))

In [217]:
precision_score(y_pred = y_pred, y_true = some_df['Target'])

1.0

In [218]:
recall_score(y_pred = y_pred, y_true = some_df['Target'])

0.98319327731092432

In [287]:
from sklearn.naive_bayes import GaussianNB

In [288]:
NBClassifer = GaussianNB()
NBClassifer.fit(some_df.drop('Target', 1), some_df['Target'])
y_pred = NBClassifer.predict(some_df.drop('Target', 1))

In [289]:
precision_score(y_pred = y_pred, y_true = some_df['Target'])

0.0032446463335496431

In [290]:
recall_score(y_pred = y_pred, y_true = some_df['Target'])

0.88235294117647056

In [291]:
confusion_matrix(y_pred = y_pred, y_true = some_df['Target'])

array([[54341, 32256],
       [   14,   105]])

In [167]:
from sklearn.tree import export_graphviz

In [181]:
export_graphviz(tree_for_training_data, out_file='tree.dot', feature_names=detector_names[1:])

In [300]:
for org_number in range(20, 30):
    some_df = read_training_org_data(org_number, 5, 33)
    Z12_x, Z3_x, Z12_y, Z3_y, Ones_x, Ones_y, some_scaler = split_training_data(some_df)
    np.savetxt("Z12_x_{}.csv".format(org_number), Z12_x, delimiter = ",")
    np.savetxt("Z12_y_{}.csv".format(org_number), Z12_y, delimiter = ",")
    np.savetxt("Z3_x_{}.csv".format(org_number), Z3_x, delimiter = ",")
    np.savetxt("Z3_y_{}.csv".format(org_number), Z3_y, delimiter = ",")
    np.savetxt("Ones_x_{}.csv".format(org_number), Ones_x, delimiter = ",")
    np.savetxt("Ones_y_{}.csv".format(org_number), Ones_y, delimiter = ",")
    some_test_df = read_org_test_data(org_number, 34, 52)
    T_x, T_y, T_generated_attributes = split_test_data(some_test_df, some_scaler)
    print("Number of ones in this organization number ({}) is {}".format(org_number, T_y.sum()))
    np.savetxt("T_x_{}.csv".format(org_number), T_x, delimiter = ",")
    np.savetxt("T_y_{}.csv".format(org_number), T_y, delimiter = ",")
    

Number of ones in this organization number (20) is 45
Number of ones in this organization number (21) is 52
Number of ones in this organization number (22) is 42
Number of ones in this organization number (23) is 35
Number of ones in this organization number (24) is 46
Number of ones in this organization number (25) is 44
Number of ones in this organization number (26) is 40
Number of ones in this organization number (27) is 47
Number of ones in this organization number (28) is 46
Number of ones in this organization number (29) is 41


In [301]:
for org_number in range(30, 40):
    some_df = read_training_org_data(org_number, 5, 33)
    Z12_x, Z3_x, Z12_y, Z3_y, Ones_x, Ones_y, some_scaler = split_training_data(some_df)
    np.savetxt("Z12_x_{}.csv".format(org_number), Z12_x, delimiter = ",")
    np.savetxt("Z12_y_{}.csv".format(org_number), Z12_y, delimiter = ",")
    np.savetxt("Z3_x_{}.csv".format(org_number), Z3_x, delimiter = ",")
    np.savetxt("Z3_y_{}.csv".format(org_number), Z3_y, delimiter = ",")
    np.savetxt("Ones_x_{}.csv".format(org_number), Ones_x, delimiter = ",")
    np.savetxt("Ones_y_{}.csv".format(org_number), Ones_y, delimiter = ",")
    some_test_df = read_org_test_data(org_number, 34, 52)
    T_x, T_y, T_generated_attributes = split_test_data(some_test_df, some_scaler)
    print("Number of ones in this organization number ({}) is {}".format(org_number, T_y.sum()))
    np.savetxt("T_x_{}.csv".format(org_number), T_x, delimiter = ",")
    np.savetxt("T_y_{}.csv".format(org_number), T_y, delimiter = ",")
    

Number of ones in this organization number (30) is 50
Number of ones in this organization number (31) is 40
Number of ones in this organization number (32) is 42
Number of ones in this organization number (33) is 41
Number of ones in this organization number (34) is 56
Number of ones in this organization number (35) is 53
Number of ones in this organization number (36) is 46
Number of ones in this organization number (37) is 40
Number of ones in this organization number (38) is 42
Number of ones in this organization number (39) is 53


In [101]:
np.savetxt("Z12_x.csv", Z12_x, delimiter = ",")
np.savetxt("Z12_y.csv", Z12_y, delimiter = ",")
np.savetxt("Z3_x.csv", Z3_x, delimiter = ",")
np.savetxt("Z3_y.csv", Z3_y, delimiter = ",")
np.savetxt("Ones_x.csv", Ones_x, delimiter = ",")
np.savetxt("Ones_y.csv", Ones_y, delimiter = ",")

In [27]:
import os
os.getcwd()

'C:\\Users\\Mimran\\OneDrive - George Mason University\\C4I PC Backup\\SCITE\\RCPs Fifth Quarter\\RCP13\\Dev'

In [96]:
best_k = RCP13_Algorithm_1(Z12_x, Z3_x, Z12_y, Z3_y, Ones_x, Ones_y)

2
current area is 0.0034713597500924697
k 2 had an average area of 0.0034713597500924697
3
current area is 0.0037417372984838394
k 3 had an average area of 0.0037417372984838394
4
current area is 0.00461559416491489
k 4 had an average area of 0.00461559416491489
5
current area is 0.0044608715407439044
k 5 had an average area of 0.0044608715407439044
6
current area is 0.0033825617455157855
k 6 had an average area of 0.0033825617455157855
7
current area is 0.006033464331030584
k 7 had an average area of 0.006033464331030584
8
current area is 0.005273188876478631
k 8 had an average area of 0.005273188876478631
9
current area is 0.004818768643709578
k 9 had an average area of 0.004818768643709578
10
current area is 0.005771357957589945
k 10 had an average area of 0.005771357957589945
11
current area is 0.007996219161761055
k 11 had an average area of 0.007996219161761055
12
current area is 0.0038127286188362566
k 12 had an average area of 0.0038127286188362566
13
current area is 0.00871436

In [97]:
best_k

22

In [37]:
def get_testing_week_data_filename_QD(week_number, simulated_org_number):
    head_folder_name = "C:/Users/Mimran/Google Drive/RCP 13 and RCP14/RCP13 and RCP14/July 7 Testing Data 50 Organizations"
    full_filename = "{}/{}/org ({}).csv".format(head_folder_name, 
                                                     week_number, 
                                                     simulated_org_number)
    return full_filename

In [38]:
get_testing_week_data_filename_QD(34, 4)

'C:/Users/Mimran/Google Drive/RCP 13 and RCP14/RCP13 and RCP14/July 7 Testing Data 50 Organizations/34/org (4).csv'

In [39]:
def read_testing_week_data_QD(week_number, simulated_org_number):
    full_filename = get_testing_week_data_filename_QD(week_number, simulated_org_number)
    week_df = pd.read_csv(full_filename, usecols=list(range(1,67))) # Note this assumes similar order of users everywhere
    week_df.replace([-np.inf,np.inf], np.nan, inplace=True) #(no matching of users is necessary *under this assumption*)
    return week_df.dropna()

In [40]:
def read_org_test_data(simulated_org_number, first_week, last_week):
    all_full_week_dfs = []
    for current_week in range(first_week, last_week + 1):
        current_week_df = read_testing_week_data_QD(current_week, simulated_org_number)
        detector_string = 'X021f'
        current_week_df['trait_4'] = (current_week_df['{}'.format(detector_string)] > np.percentile(current_week_df['{}'.format(detector_string)], 90))
        detector_string = 'X021h'
        current_week_df['trait_6'] = (current_week_df['{}'.format(detector_string)] > np.percentile(current_week_df['{}'.format(detector_string)], 90))
        detector_string = 'X022f'
        current_week_df['trait_8'] = (current_week_df['{}'.format(detector_string)] > np.percentile(current_week_df['{}'.format(detector_string)], 90))
        detector_string = 'X022h'
        current_week_df['trait_10'] = (current_week_df['{}'.format(detector_string)] > np.percentile(current_week_df['{}'.format(detector_string)], 90))
        detector_string = 'X027f'
        current_week_df['trait_12'] = (current_week_df['{}'.format(detector_string)] > np.percentile(current_week_df['{}'.format(detector_string)], 90))
        detector_string = 'X027h'
        current_week_df['trait_14'] = (current_week_df['{}'.format(detector_string)] > np.percentile(current_week_df['{}'.format(detector_string)], 90))
        detector_string = 'X028f'
        current_week_df['trait_16'] = (current_week_df['{}'.format(detector_string)] > np.percentile(current_week_df['{}'.format(detector_string)], 90))
        detector_string = 'X028h'
        current_week_df['trait_18'] = (current_week_df['{}'.format(detector_string)] > np.percentile(current_week_df['{}'.format(detector_string)], 90))
        detector_string = 'X058a'
        current_week_df['trait_20'] = (current_week_df['{}'.format(detector_string)])
        all_full_week_dfs.append(current_week_df)
    return pd.concat(all_full_week_dfs)

In [41]:
all_trait_names = ["trait_" + str(trait_num) for trait_num in range(4, 21, 2)]

In [42]:
def split_test_data(sample_test_data_df, scaler_from_training_data):
    T_x, T_y, T_generated_attributes = (sample_test_data_df[detector_names[1:]], 
                                        sample_test_data_df['Target'], 
                                        sample_test_data_df[all_trait_names])
    T_x = scaler_from_training_data.transform(T_x)
    return T_x, T_y.values, T_generated_attributes

In [31]:
from sklearn.metrics import mean_squared_error

In [32]:
def get_mse(true_detector_values, predicted_detector_values):
    return mean_squared_error(true_detector_values, predicted_detector_values)

In [33]:
def predict_using_mse(true_detector_values, predicted_detector_values, test_cutoff):
    return (mean_squared_error(true_detector_values, predicted_detector_values) > test_cutoff).astype(int)

In [34]:
def calc_f1_score_using_threshold(actual_y, true_detector_values, predicted_detector_values, test_cutoff):
    return f1_score(actual_y, predict_using_mse(true_detector_values, predicted_detector_values, test_cutoff))

In [172]:
def RCP13_Algorithm_2(Z12_x, Z3_x, Z12_y, Z3_y, Ones_x, Ones_y, T_x, T_y,  
                      T_generated_attributes, chosen_k, num_iterations = 100):
    answer_dict = {"Answer_" + str(answer_num) : np.zeros(num_iterations) for answer_num in range(1, 22)}
    for iteration_num in range(num_iterations):
        n_net = MLPRegressor(hidden_layer_sizes=(chosen_k,), 
                                  activation = 'tanh', 
                                  solver = 'sgd', max_iter = 250, early_stopping = True)
        n_net.fit(Z12_x, Z12_y)
        model_prediction_output = n_net.predict(np.vstack([Z3_x, Ones_x]))
        def func_to_min(x):
            return -calc_f1_score_using_threshold(x, np.hstack([Z3_y, Ones_y]), model_prediction_output)
        chosen_tau = minimize_scalar(func_to_min, bounds = (0, 1), method = 'bounded').x
        print("optimized cutoff is {}".format(chosen_tau))
        prediction_output_for_test_data = n_net.predict(T_x)
        T_labels = (prediction_output_for_test_data > chosen_tau).astype(int)
        answer_dict["Answer_1"][iteration_num] = (T_y & T_labels).sum() / T_y.sum()
        answer_dict["Answer_2"][iteration_num] = (T_y & T_labels).sum() / T_labels.sum()
        answer_dict["Answer_3"][iteration_num] = (T_y & T_labels).sum() / (T_y ^ 1).sum()
        answer_dict["Answer_4"][iteration_num] = (T_generated_attributes['trait_4'].values & T_labels).mean() / (T_labels).mean()
        answer_dict["Answer_5"][iteration_num] = (T_generated_attributes['trait_4'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_4'])
        answer_dict["Answer_6"][iteration_num] = (T_generated_attributes['trait_6'].values & T_labels).mean() / (T_labels).mean()
        answer_dict["Answer_7"][iteration_num] = (T_generated_attributes['trait_6'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_6'])
        answer_dict["Answer_8"][iteration_num] = (T_generated_attributes['trait_8'].values & T_labels).mean() / (T_labels).mean()
        answer_dict["Answer_9"][iteration_num] = (T_generated_attributes['trait_8'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_8'])
        answer_dict["Answer_10"][iteration_num] = (T_generated_attributes['trait_10'].values & T_labels).mean() / (T_labels).mean()
        answer_dict["Answer_11"][iteration_num] = (T_generated_attributes['trait_10'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_10'])
        answer_dict["Answer_12"][iteration_num] = (T_generated_attributes['trait_12'].values & T_labels).mean() / (T_labels).mean()
        answer_dict["Answer_13"][iteration_num] = (T_generated_attributes['trait_12'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_12'])
        answer_dict["Answer_14"][iteration_num] = (T_generated_attributes['trait_14'].values & T_labels).mean() / (T_labels).mean()
        answer_dict["Answer_15"][iteration_num] = (T_generated_attributes['trait_14'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_14'])
        answer_dict["Answer_16"][iteration_num] = (T_generated_attributes['trait_16'].values & T_labels).mean() / (T_labels).mean()
        answer_dict["Answer_17"][iteration_num] = (T_generated_attributes['trait_16'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_16'])
        answer_dict["Answer_18"][iteration_num] = (T_generated_attributes['trait_18'].values & T_labels).mean() / (T_labels).mean()
        answer_dict["Answer_19"][iteration_num] = (T_generated_attributes['trait_18'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_18'])
        answer_dict["Answer_20"][iteration_num] = (T_generated_attributes['trait_20'].values & T_labels).mean() / (T_labels).mean()
        answer_dict["Answer_21"][iteration_num] = (T_generated_attributes['trait_20'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_20'])
    return answer_dict

In [266]:
some_test_df = read_org_test_data(9, 34, 52)

In [267]:
T_x, T_y, T_generated_attributes = split_test_data(some_test_df, some_scaler)

In [268]:
smartClassifier.score(T_x, T_y)

0.99925224750607544

In [283]:
precision_score(y_pred = tree_for_training_data.predict(T_x), y_true=T_y)

0.00073174301185423675

In [284]:
recall_score(y_pred = tree_for_training_data.predict(T_x), y_true=T_y)

0.22727272727272727

In [286]:
tree_for_training_data.predict(T_x).mean()

0.23224512686300836

In [281]:
smartClassifier.predict(T_x).mean()

0.0

In [272]:
T_y.sum()

44

In [105]:
np.savetxt("T_x.csv", T_x, delimiter = ",")
np.savetxt("T_y.csv", T_y, delimiter = ",")

In [106]:
T_labels = pd.read_csv("T_labels.csv")

In [107]:
T_labels = (T_labels.values).T

In [108]:
T_labels

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [109]:
T_labels.sum()

62

In [110]:
answer_dict = {"Answer_" + str(answer_num) : np.zeros(1) for answer_num in range(1, 22)}
iteration_num = 0

In [111]:
answer_dict["Answer_1"][iteration_num] = (T_y & T_labels).sum() / T_y.sum()
answer_dict["Answer_2"][iteration_num] = (T_y & T_labels).sum() / T_labels.sum()
answer_dict["Answer_3"][iteration_num] = (T_y & T_labels).sum() / (T_y ^ 1).sum()
answer_dict["Answer_4"][iteration_num] = (T_generated_attributes['trait_4'].values & T_labels).mean() / (T_labels).mean()
answer_dict["Answer_5"][iteration_num] = (T_generated_attributes['trait_4'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_4'])
answer_dict["Answer_6"][iteration_num] = (T_generated_attributes['trait_6'].values & T_labels).mean() / (T_labels).mean()
answer_dict["Answer_7"][iteration_num] = (T_generated_attributes['trait_6'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_6'])
answer_dict["Answer_8"][iteration_num] = (T_generated_attributes['trait_8'].values & T_labels).mean() / (T_labels).mean()
answer_dict["Answer_9"][iteration_num] = (T_generated_attributes['trait_8'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_8'])
answer_dict["Answer_10"][iteration_num] = (T_generated_attributes['trait_10'].values & T_labels).mean() / (T_labels).mean()
answer_dict["Answer_11"][iteration_num] = (T_generated_attributes['trait_10'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_10'])
answer_dict["Answer_12"][iteration_num] = (T_generated_attributes['trait_12'].values & T_labels).mean() / (T_labels).mean()
answer_dict["Answer_13"][iteration_num] = (T_generated_attributes['trait_12'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_12'])
answer_dict["Answer_14"][iteration_num] = (T_generated_attributes['trait_14'].values & T_labels).mean() / (T_labels).mean()
answer_dict["Answer_15"][iteration_num] = (T_generated_attributes['trait_14'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_14'])
answer_dict["Answer_16"][iteration_num] = (T_generated_attributes['trait_16'].values & T_labels).mean() / (T_labels).mean()
answer_dict["Answer_17"][iteration_num] = (T_generated_attributes['trait_16'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_16'])
answer_dict["Answer_18"][iteration_num] = (T_generated_attributes['trait_18'].values & T_labels).mean() / (T_labels).mean()
answer_dict["Answer_19"][iteration_num] = (T_generated_attributes['trait_18'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_18'])
answer_dict["Answer_20"][iteration_num] = (T_generated_attributes['trait_20'].values & T_labels).mean() / (T_labels).mean()
answer_dict["Answer_21"][iteration_num] = (T_generated_attributes['trait_20'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_20'])

In [112]:
answer_dict

{'Answer_1': array([ 0.]),
 'Answer_10': array([ 0.58064516]),
 'Answer_11': array([ 0.00617072]),
 'Answer_12': array([ 0.48387097]),
 'Answer_13': array([ 0.00588813]),
 'Answer_14': array([ 0.51612903]),
 'Answer_15': array([ 0.00560224]),
 'Answer_16': array([ 0.5]),
 'Answer_17': array([ 0.00598918]),
 'Answer_18': array([ 0.56451613]),
 'Answer_19': array([ 0.00622444]),
 'Answer_2': array([ 0.]),
 'Answer_20': array([ 0.08064516]),
 'Answer_21': array([ 0.01010101]),
 'Answer_3': array([ 0.]),
 'Answer_4': array([ 0.51612903]),
 'Answer_5': array([ 0.00562291]),
 'Answer_6': array([ 0.51612903]),
 'Answer_7': array([ 0.00549828]),
 'Answer_8': array([ 0.53225806]),
 'Answer_9': array([ 0.00590129])}

In [310]:
T_labels_dict = {}
T_labels_dict[20] = pd.read_csv("T_labels20.csv").values.T
some_test_df = read_org_test_data(20, 34, 52)
T_x, T_y, T_generated_attributes = split_test_data(some_test_df, some_scaler)
answer_dict = {"Answer_" + str(answer_num) : np.zeros(1) for answer_num in range(1, 22)}
T_labels = T_labels_dict[20]
iteration_num = 0

In [311]:
answer_dict["Answer_1"][iteration_num] = (T_y & T_labels).sum() / T_y.sum()
answer_dict["Answer_2"][iteration_num] = (T_y & T_labels).sum() / T_labels.sum()
answer_dict["Answer_3"][iteration_num] = (T_y & T_labels).sum() / (T_y ^ 1).sum()
answer_dict["Answer_4"][iteration_num] = (T_generated_attributes['trait_4'].values & T_labels).mean() / (T_labels).mean()
answer_dict["Answer_5"][iteration_num] = (T_generated_attributes['trait_4'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_4'])
answer_dict["Answer_6"][iteration_num] = (T_generated_attributes['trait_6'].values & T_labels).mean() / (T_labels).mean()
answer_dict["Answer_7"][iteration_num] = (T_generated_attributes['trait_6'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_6'])
answer_dict["Answer_8"][iteration_num] = (T_generated_attributes['trait_8'].values & T_labels).mean() / (T_labels).mean()
answer_dict["Answer_9"][iteration_num] = (T_generated_attributes['trait_8'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_8'])
answer_dict["Answer_10"][iteration_num] = (T_generated_attributes['trait_10'].values & T_labels).mean() / (T_labels).mean()
answer_dict["Answer_11"][iteration_num] = (T_generated_attributes['trait_10'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_10'])
answer_dict["Answer_12"][iteration_num] = (T_generated_attributes['trait_12'].values & T_labels).mean() / (T_labels).mean()
answer_dict["Answer_13"][iteration_num] = (T_generated_attributes['trait_12'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_12'])
answer_dict["Answer_14"][iteration_num] = (T_generated_attributes['trait_14'].values & T_labels).mean() / (T_labels).mean()
answer_dict["Answer_15"][iteration_num] = (T_generated_attributes['trait_14'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_14'])
answer_dict["Answer_16"][iteration_num] = (T_generated_attributes['trait_16'].values & T_labels).mean() / (T_labels).mean()
answer_dict["Answer_17"][iteration_num] = (T_generated_attributes['trait_16'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_16'])
answer_dict["Answer_18"][iteration_num] = (T_generated_attributes['trait_18'].values & T_labels).mean() / (T_labels).mean()
answer_dict["Answer_19"][iteration_num] = (T_generated_attributes['trait_18'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_18'])
answer_dict["Answer_20"][iteration_num] = (T_generated_attributes['trait_20'].values & T_labels).mean() / (T_labels).mean()
answer_dict["Answer_21"][iteration_num] = (T_generated_attributes['trait_20'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_20'])

In [316]:
answer_dict

{'Answer_1': array([ 0.]),
 'Answer_10': array([ 0.12837838]),
 'Answer_11': array([ 0.00325621]),
 'Answer_12': array([ 0.08108108]),
 'Answer_13': array([ 0.00242915]),
 'Answer_14': array([ 0.12162162]),
 'Answer_15': array([ 0.00320171]),
 'Answer_16': array([ 0.10810811]),
 'Answer_17': array([ 0.00317965]),
 'Answer_18': array([ 0.12162162]),
 'Answer_19': array([ 0.00318528]),
 'Answer_2': array([ 0.]),
 'Answer_20': array([ 0.05405405]),
 'Answer_21': array([ 0.01410935]),
 'Answer_3': array([ 0.]),
 'Answer_4': array([ 0.10135135]),
 'Answer_5': array([ 0.0026325]),
 'Answer_6': array([ 0.12162162]),
 'Answer_7': array([ 0.00309332]),
 'Answer_8': array([ 0.08783784]),
 'Answer_9': array([ 0.00230906])}

In [321]:
pd.DataFrame(answer_dict).to_csv("Answers_20.csv")

In [318]:
for answer_num in range(1, 22):
    print("{} \t {:.3f}".format("Answer_{}".format(answer_num), answer_dict["Answer_{}".format(answer_num)][0]))

Answer_1 	 0.000
Answer_2 	 0.000
Answer_3 	 0.000
Answer_4 	 0.101
Answer_5 	 0.003
Answer_6 	 0.122
Answer_7 	 0.003
Answer_8 	 0.088
Answer_9 	 0.002
Answer_10 	 0.128
Answer_11 	 0.003
Answer_12 	 0.081
Answer_13 	 0.002
Answer_14 	 0.122
Answer_15 	 0.003
Answer_16 	 0.108
Answer_17 	 0.003
Answer_18 	 0.122
Answer_19 	 0.003
Answer_20 	 0.054
Answer_21 	 0.014


In [319]:
successes = [20, 21, 24, 27, 28, 30, 34, 35, 36, 39]

In [335]:
T_labels_dict = {}
answer_dicts = {}
for org_number in [39]:
    T_labels_dict[org_number] = pd.read_csv("T_labels{}.csv".format(org_number)).values.T
    some_test_df = read_org_test_data(org_number, 34, 52)
    T_x, T_y, T_generated_attributes = split_test_data(some_test_df, some_scaler)
    answer_dict = {"Answer_" + str(answer_num) : np.zeros(1) for answer_num in range(1, 22)}
    T_labels = T_labels_dict[org_number]
    iteration_num = 0
    answer_dict["Answer_1"][iteration_num] = (T_y & T_labels).sum() / T_y.sum()
    answer_dict["Answer_2"][iteration_num] = (T_y & T_labels).sum() / T_labels.sum()
    answer_dict["Answer_3"][iteration_num] = (T_y & T_labels).sum() / (T_y ^ 1).sum()
    answer_dict["Answer_4"][iteration_num] = (T_generated_attributes['trait_4'].values & T_labels).mean() / (T_labels).mean()
    answer_dict["Answer_5"][iteration_num] = (T_generated_attributes['trait_4'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_4'])
    answer_dict["Answer_6"][iteration_num] = (T_generated_attributes['trait_6'].values & T_labels).mean() / (T_labels).mean()
    answer_dict["Answer_7"][iteration_num] = (T_generated_attributes['trait_6'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_6'])
    answer_dict["Answer_8"][iteration_num] = (T_generated_attributes['trait_8'].values & T_labels).mean() / (T_labels).mean()
    answer_dict["Answer_9"][iteration_num] = (T_generated_attributes['trait_8'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_8'])
    answer_dict["Answer_10"][iteration_num] = (T_generated_attributes['trait_10'].values & T_labels).mean() / (T_labels).mean()
    answer_dict["Answer_11"][iteration_num] = (T_generated_attributes['trait_10'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_10'])
    answer_dict["Answer_12"][iteration_num] = (T_generated_attributes['trait_12'].values & T_labels).mean() / (T_labels).mean()
    answer_dict["Answer_13"][iteration_num] = (T_generated_attributes['trait_12'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_12'])
    answer_dict["Answer_14"][iteration_num] = (T_generated_attributes['trait_14'].values & T_labels).mean() / (T_labels).mean()
    answer_dict["Answer_15"][iteration_num] = (T_generated_attributes['trait_14'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_14'])
    answer_dict["Answer_16"][iteration_num] = (T_generated_attributes['trait_16'].values & T_labels).mean() / (T_labels).mean()
    answer_dict["Answer_17"][iteration_num] = (T_generated_attributes['trait_16'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_16'])
    answer_dict["Answer_18"][iteration_num] = (T_generated_attributes['trait_18'].values & T_labels).mean() / (T_labels).mean()
    answer_dict["Answer_19"][iteration_num] = (T_generated_attributes['trait_18'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_18'])
    answer_dict["Answer_20"][iteration_num] = (T_generated_attributes['trait_20'].values & T_labels).mean() / (T_labels).mean()
    answer_dict["Answer_21"][iteration_num] = (T_generated_attributes['trait_20'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_20'])
    for answer_num in range(1, 22):
        answer_dict["Answer_{}".format(answer_num)] = answer_dict["Answer_{}".format(answer_num)][0]
    answer_dicts[org_number] = answer_dict

In [337]:
pd.DataFrame(answer_dicts).T

,Answer_1,Answer_10,Answer_11,Answer_12,Answer_13,Answer_14,Answer_15,Answer_16,Answer_17,Answer_18,...,Answer_2,Answer_20,Answer_21,Answer_3,Answer_4,Answer_5,Answer_6,Answer_7,Answer_8,Answer_9
39,0.226415,0.08545,0.012774,0.073903,0.012942,0.099307,0.015019,0.06582,0.0114,0.08545,...,0.013857,0.360277,0.6,0.000204,0.084296,0.012843,0.095843,0.0142,0.076212,0.011665


In [338]:
(pd.DataFrame(answer_dicts).T).to_csv("Attempt_at_answer_39.csv")